# Set up all needed functions to enable load test

- create workspace
- enable cbas
- get wdas and cbas urls from Leo
- upload data to wds
- trigger a workflow in cbas (currently hardcoded to assemble_refbased but can be easily changed)

The following information needs to be populated prior to executing notebook: 
- your azure token to allow execution
- name of bee where the load test will run
- name of billing project where the load test will run
- number of workspace the load test should create
- if cbas should we turned on for created workspaces (True=on, False=off)
- if wds upload should be triggered
- if cbas workflow should be triggered and on how many workspaces (cant be larger then number of workspaces that the test will create)


**TODO: First make sure to fill the data in the first section below! Then you can execute the rest without changes.**

In [38]:
# Setup configuration
azure_token = "Bearer eyJhbGciOiJSUzI1NiIsImtpZCI6Inp4UnJ6aEw4OTBrYi1vVkZOWkdWUFlxSzYtSkI3RFRlTGVvMllTbzRtc1UiLCJ0eXAiOiJKV1QifQ.eyJlbWFpbCI6Inl1bGlhbG92ZXN0ZXJyYUBnbWFpbC5jb20iLCJnaXZlbl9uYW1lIjoiWXVsaWEiLCJmYW1pbHlfbmFtZSI6IkR1YiIsIm5hbWUiOiJZdWxpYSBEdWIiLCJpZHAiOiJnb29nbGUuY29tIiwiaWRwX2FjY2Vzc190b2tlbiI6InlhMjkuYTBBZkJfYnlETnFQUTdIdmtIQzM2WXNuLU5KS25Cd1EwZWRQQ0ZJYmZzQ3lwMnpxY05hMEFGUTJXcTBkZVBJQThQVGpDN2g4WTEwOVQ0ZzRBazZEVlNwZ1ZQeEtPYWVsT3NvUVF6VS1Hb3FWQlZEczJiS3F3STN4YVVwc1VuYXR5TWFfc2RZZTFOT2dBdFJ6WHdabmpDa0YzUUszZV8yOW9hQ2dZS0FiMFNBUkFTRlFIc3ZZbHMxTWxLRWJuWUNaSEFvWkVYdjFPZ0RBMDE2NiIsImdvb2dsZV9pZCI6IjEwMzU0ODU0NTQwNzgxNDk2MzkxMSIsInBpY3R1cmUiOiJodHRwczovL2xoMy5nb29nbGV1c2VyY29udGVudC5jb20vYS9BQWNIVHRjeUdMWTJBUWYwbV9jd3dnYTBzdHpENktURXE0dEkzSXNoTEVfaExKTms9czk2LWMiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwic3ViIjoiZGEyNDVlNGUtNGUzNC00NzQ4LTg4OTAtMDc1YWNjNDBlZTI5IiwidGlkIjoiZmQwYmMwZWYtMTc0Ny00ZWU2LWFiM2UtZDRkNmJiODgyZDQwIiwiYXpwIjoiYmJkMDdkNDMtMDFjYi00YjY5LThmZDAtNTc0NmQ5YTVjOWZlIiwidmVyIjoiMS4wIiwiaWF0IjoxNjkxNjk2Mzg1LCJhdWQiOiJiYmQwN2Q0My0wMWNiLTRiNjktOGZkMC01NzQ2ZDlhNWM5ZmUiLCJleHAiOjE2OTE2OTk5ODUsImlzcyI6Imh0dHBzOi8vdGVycmFkZXZiMmMuYjJjbG9naW4uY29tL2ZkMGJjMGVmLTE3NDctNGVlNi1hYjNlLWQ0ZDZiYjg4MmQ0MC92Mi4wLyIsIm5iZiI6MTY5MTY5NjM4NX0.PMbUFw3iqZhFj173nIGy5O6MyVfilZ602VPZ43LfloEhU2OQPY1rACwC72T-eVP7YStLbR_FH-yhkytmdN9K449-bUQq_gkiHxI6cCG8oDo3R-LNAPrhkt-8rCzN2uQyVY3dCLQbky4wJ7MFrmEO-g4UhR1hmAibTOeLMTBqpbCDWS1yrtZxzDIsK_QxGZSV6vch9GYrMdKg0ixGcuIl3403R0CPgH91nNSvN8qnSgd6Y_ZvK3q9I1nMGnH-Fsc4MIy3I3DyMw6tDm3jrCtSm4eJuNFTHR7cxqLP8gdqZ99HkJyxXcretaolo6jbBC5wur2Av0ry2R-hIxljn70A5g"
bee_name = "yuliadub-load-test"; 
billing_project_name = "yulia-load-test"; 
number_of_workspaces = 1;
cbas=True;
wds_upload=True
cbas_submit_workflow=True
number_of_workflow_to_kick_off = 1

In [39]:
import sys
import os
import json
import requests
import io
import uuid
import random
import string
!pip install wds-client
import wds_client
import requests
import time

# define major service endpoints based on bee name
workspace_manager_url = f"https://workspace.{bee_name}.bee.envs-terra.bio"
rawls_url = f"https://rawls.{bee_name}.bee.envs-terra.bio"
leo_url = f"https://leonardo.{bee_name}.bee.envs-terra.bio"

# CREATE WORKSPACE ACTION
def create_workspace(cbas):
    # create a new workspace, need to have attributes or api call doesnt work
    api_call2 = f"{rawls_url}/api/workspaces";
    request_body= {
      "namespace": billing_project_name, # Billing project name
      "name": f"api-workspace-{''.join(random.choices(string.ascii_lowercase, k=5))}", # workspace name
      "attributes": {}};
    
    response = requests.post(url=api_call2, json=request_body, headers=header)
    
    #example json that is returned by request: 'attributes': {}, 'authorizationDomain': [], 'bucketName': '', 'createdBy': 'yulialovesterra@gmail.com', 'createdDate': '2023-08-03T20:10:59.116Z', 'googleProject': '', 'isLocked': False, 'lastModified': '2023-08-03T20:10:59.116Z', 'name': 'api-workspace-1', 'namespace': 'yuliadub-test2', 'workspaceId': 'ac466322-2325-4f57-895d-fdd6c3f8c7ad', 'workspaceType': 'mc', 'workspaceVersion': 'v2'}
    json2 = response.json()
    data = json.loads(json.dumps(json2))
    
    print(data['workspaceId'])
    
    # enable CBAS if specified
    if cbas is True:
        print(f"Enabling CBAS for workspace {data['workspaceId']}")
        api_call3 = f"{leo_url}/api/apps/v2/{data['workspaceId']}/terra-app-{str(uuid.uuid4())}";
        request_body2 = {
          "appType": "CROMWELL"
        } 
        
        response = requests.post(url=api_call3, json=request_body2, headers=header)
        # will return 202 or error
        print(response)

    return data['workspaceId']

# GET WDS OR CROMWELL ENDPOINT URL FROM LEO
def get_app_url(workspaceId, app):
    """"Get url for wds/cbas."""
    uri = f"{leo_url}/api/apps/v2/{workspaceId}?includeDeleted=false"

    headers = {"Authorization": azure_token,
               "accept": "application/json"}

    response = requests.get(uri, headers=headers)
    status_code = response.status_code

    if status_code != 200:
        return response.text
    print(f"Successfully retrieved details.")
    response = json.loads(response.text)

    app_url = ""
    app_type = "CROMWELL" if app != 'wds' else app.upper();
    print(f"App type: {app_type}")
    for entries in response: 
        if entries['appType'] == app_type and entries['proxyUrls'][app] is not None:
            print(entries['status'])
            if(entries['status'] == "PROVISIONING"):
                print(f"{app} is still provisioning")
                break
            print(f"App status: {entries['status']}")
            app_url = entries['proxyUrls'][app]
            break 

    if app_url is None: 
        print(f"{app} is missing in current workspace")
    else:
        print(f"{app} url: {app_url}")

    return app_url

# UPLOAD DATA TO WORSPACE DATA SERVICE IN A WORKSPACE
def upload_wds_data(wds_url, current_workspaceId, tsv_file_name, recordName):
    version="v0.2"
    api_client = wds_client.ApiClient(header_name='Authorization', header_value=azure_token)
    api_client.configuration.host = wds_url
    
    # records client is used to interact with Records in the data table
    records_client = wds_client.RecordsApi(api_client)
    # data to upload to wds table
    
    # Upload entity to workspace data table with name "testType_uploaded"
    response = records_client.upload_tsv(current_workspaceId, version, recordName, tsv_file_name)
    print(response)

# KICK OFF A WORKFLOW INSIDE A WORKSPACE
def submit_workflow_assemble_refbased(workspaceId, dataFile):
    cbas_url = get_app_url(workspaceId, "cbas")
    print(cbas_url)
    #open text file in read mode
    text_file = open(dataFile, "r")
    request_body2 = text_file.read();
    text_file.close()
    
    uri = f"{cbas_url}/api/batch/v1/run_sets"
    
    headers = {"Authorization": azure_token,
               "accept": "application/json", 
              "Content-Type": "application/json"}
    
    response = requests.post(uri, data=request_body2, headers=headers)
    # example of what it returns: {'run_set_id': 'cdcdc570-f6f3-4425-9404-4d70cd74ce2a', 'runs': [{'run_id': '0a72f308-4931-436b-bbfe-55856f7c1a39', 'state': 'UNKNOWN', 'errors': 'null'}, {'run_id': 'eb400221-efd7-4e1a-90c9-952f32a10b60', 'state': 'UNKNOWN', 'errors': 'null'}], 'state': 'RUNNING'}
    print(response.json())

Defaulting to user installation because normal site-packages is not writeable


# This section executes the load test. 

Unless you want to change the behavior of the test, no need to make any changes to this section - output is mostly for debugging reason to make sure things are moving along. 

In [ ]:
# get existing workspaces
api_call1 = f"{workspace_manager_url}/api/workspaces/v1";
header = {"Authorization": azure_token};

#response = requests.get(url=api_call1, headers=header)
#print(response.json())

workspaceIds = []

while number_of_workspaces != 0:
    # create the specified number os workspaces
    workspaceId = create_workspace(cbas)
    workspaceIds.append(workspaceId)
    number_of_workspaces-=1


# track to see when the workspace WDS are ready to upload data into them 
# sleep for 5 minutes to allow WDS to start up, if no wds, only sleep 2 minutes to let cbas start up
if wds_upload:
    time.sleep(300)
elif cbas_submit_workflow:
    time.sleep(120) 
else:
    print("LOAD TEST COMPLETE.")

if wds_upload:
    for workspace in workspaceIds:
        print(f"trying to see wds is ready to upload stuff for workspace {workspace}")
        wds_url = get_app_url(workspace, "wds")
        if wds_url == "":
            print(f"wds not ready or errored out for workspace {workspace}")
            continue
        upload_wds_data(wds_url, workspace, "resources/test.tsv", "test")

if cbas_submit_workflow:
    # next trigger a workflow in each of the workspaces, at this time this doesnt monitor if this was succesful or not
    # upload file needed for workflow to run
    upload_wds_data(wds_url, workspace, "resources/sraloadtest.tsv", "sraloadtest")
    for workspace in workspaceIds:
        if number_of_workflow_to_kick_off != 0:
            submit_workflow_assemble_refbased(workspace, "resources/assemble_refbased.json")
            number_of_workflow_to_kick_off -=1 
        else:
            break

print("LOAD TEST COMPLETE.")
